In [1]:
import pandas as pd
from sqlalchemy import create_engine
#config for password
from config import password

In [2]:
# Read our city data into pandas

city_file = "resources/cities.csv"

city_df = pd.read_csv(city_file)
city_df.head()

,id,name,coords,start_year,url_name,country,country_state
0,5,Aberdeen,POINT(-2.15 57.15),2017,aberdeen,Scotland,NaN
1,6,Adelaide,POINT(138.6 -34.91666667),2017,adelaide,Australia,NaN
2,7,Algiers,POINT(3 36.83333333),2017,algiers,Algeria,NaN
3,9,Ankara,POINT(32.91666667 39.91666667),2017,ankara,Turkey,NaN
4,16,Belém,POINT(-48.48333333 -1.466666667),2017,belem,Brazil,NaN


In [3]:
# Read our station data into pandas 
station_file = "resources/stations.csv"
station_df = pd.read_csv(station_file)
station_df.head()

,id,name,geometry,buildstart,opening,closure,city_id
0,7694,Keisei Tsudanuma,POINT(140.024812197129 35.6837744784723),1921.0,1921.0,999999.0,114
1,6003,Kossuth Lajos tér,POINT(19.0462376564033 47.5054880717671),0.0,0.0,999999.0,29
2,7732,Saint-Charles,POINT(5.3801556 43.3024646),1973.0,1977.0,999999.0,74
3,7695,Keisei Makuhari-Hongo,POINT(140.042146725175 35.6726021159981),1991.0,1991.0,999999.0,114
4,7726,Chartreux,POINT(5.4014815 43.309129),1973.0,1977.0,999999.0,74


In [4]:
# Create a filtered dataframe from specific columns
city_cols = ["id", "name", "start_year","country"]
city_transformed= city_df[city_cols].copy()

# Rename the column headers
city_transformed = city_transformed.rename(columns={"id": "city_id",
                                                      "name": "city_name",
                                                      "start_year": "start_year",
                                                      "country":"country"})

# Clean the data by dropping duplicates and setting the index
city_transformed.drop_duplicates("city_id", inplace=True)
city_transformed.set_index("city_id", inplace=True)

city_transformed.head()

,city_name,start_year,country
city_id,,,
5,Aberdeen,2017,Scotland
6,Adelaide,2017,Australia
7,Algiers,2017,Algeria
9,Ankara,2017,Turkey
16,Belém,2017,Brazil


In [5]:
#count of city data
city_transformed.count()

city_name     337
start_year    337
country       337
dtype: int64

In [6]:
#check for missing data - looks good all rows have data!
cities_dropped = city_transformed.dropna()
cities_dropped.count()

city_name     337
start_year    337
country       337
dtype: int64

In [7]:
# Create a filtered dataframe from specific columns-stations

station_cols = ["id", "name", "buildstart", "opening","city_id"]
station_rename = station_df[station_cols].copy()

# Rename the column headers
station_rename =station_rename.rename(columns={"id": "station_id",
                                                         "name": "station_name",
                                                         "buildstart": "buildstart",
                                                         "opening": "opening",
                                                        "city_id": "city_id"})




In [8]:
# Set index
station_transformed = station_rename.copy()


In [9]:

station_transformed.set_index("station_id", inplace=True)

station_transformed.head()

,station_name,buildstart,opening,city_id
station_id,,,,
7694,Keisei Tsudanuma,1921.0,1921.0,114
6003,Kossuth Lajos tér,0.0,0.0,29
7732,Saint-Charles,1973.0,1977.0,74
7695,Keisei Makuhari-Hongo,1991.0,1991.0,114
7726,Chartreux,1973.0,1977.0,74


In [10]:
#count of city data

station_transformed.count()

station_name    14624
buildstart      14719
opening         16102
city_id         16171
dtype: int64

In [11]:
#check for missing data
station_dropped = station_transformed.dropna()
station_dropped.count()

station_name    13125
buildstart      13125
opening         13125
city_id         13125
dtype: int64

In [12]:
# Combine existing tables to create a master dataframe with all relevant data so 
# that an analyst would not need to join 2 tables in SQL 
# Merging on city_id
master_df = pd.merge(city_transformed, station_rename, on='city_id')


master_df.set_index("station_id", inplace=True)
master_df.head()

,city_id,city_name,start_year,country,station_name,buildstart,opening
station_id,,,,,,,
8421,211,Ottawa,2000,Canada,Carling,2000.0,2001.0
8422,211,Ottawa,2000,Canada,Bayview,2000.0,2001.0
6324,147,Chicago,1892,United States,Kedzie,0.0,1907.0
6317,147,Chicago,1892,United States,Irving Park,0.0,1907.0
6368,147,Chicago,1892,United States,Grand,0.0,1921.0


In [13]:
#start connection to postgres
rds_connection_string = (f'postgres:{password}@localhost:5432/transit_systems')
engine = create_engine(f'postgresql://{rds_connection_string}')

In [14]:
#Verify tables have been created
engine.table_names()

['cities', 'stations', 'master']

In [15]:
station_transformed.columns

Index(['station_name', 'buildstart', 'opening', 'city_id'], dtype='object')

In [16]:
#load dataframe into Database-cities

city_transformed.to_sql(name='cities', con=engine, if_exists='append', index=True)


In [17]:
#load dataframe into Database-stations

station_transformed.to_sql(name='stations', con=engine, if_exists='append', index=True)



In [18]:
#load dataframe into Database-stations

master_df.to_sql(name='master', con=engine, if_exists='append', index=True)